## Install and import libraries

In [1]:
!pip install yfinance pykalman -q
from pykalman import KalmanFilter
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import iplot
import yfinance as yf

### Download stock data
The code downloads historical data for the HDFC Bank (HDFCBANK) from January 1, 2020, to November 10, 2023, using Yahoo Finance's yfinance library and displays the first five rows of this dataset with data.head().

In [2]:
data = yf.download("HDFCBANK.NS", start="2020-01-01", end="2023-11-10")
data.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-01,1276.099976,1280.000000,1270.599976,1278.599976,1244.189697,1836849
2020-01-02,1279.000000,1288.000000,1279.000000,1286.750000,1252.120361,3068583
2020-01-03,1282.199951,1285.000000,1263.599976,1268.400024,1234.264160,5427775
2020-01-06,1260.000000,1261.800049,1236.000000,1240.949951,1207.552979,5445093
2020-01-07,1258.900024,1271.449951,1252.250000,1260.599976,1226.674072,7362247


### Construct Kalman Filter
This code constructs a Kalman Filter to process and smooth the adjusted closing prices of a financial asset, stored in data['Adj Close']. It sets up the filter with specific matrices and parameters to model the asset's price as a random walk, applies the filter to get smoothed state estimates, and stores these estimates as a new column KF_mean in the dataset, displaying the first five rows of the updated dataset.


In [3]:
# Construct a Kalman filter
kf = KalmanFilter(transition_matrices = [1],    # The value for At. It is a random walk so is set to 1.0
                  observation_matrices = [1],   # The value for Ht.
                  initial_state_mean = 0,       # Any initial value. It will converge to the true state value.
                  initial_state_covariance = 1, # Sigma value for the Qt in Equation (1) the Gaussian distribution
                  observation_covariance=1,     # Sigma value for the Rt in Equation (2) the Gaussian distribution
                  transition_covariance=.01)    # A small turbulence in the random walk parameter 1.0
# Get the Kalman smoothing
state_means, _ = kf.filter(data['Adj Close'].values)

# Call it KF_mean
data['KF_mean'] = np.array(state_means)
data.head()

,Open,High,Low,Close,Adj Close,Volume,KF_mean
Date,,,,,,,
2020-01-01,1276.099976,1280.000000,1270.599976,1278.599976,1244.189697,1836849,622.094849
2020-01-02,1279.000000,1288.000000,1279.000000,1286.750000,1252.120361,3068583,834.884922
2020-01-03,1282.199951,1285.000000,1263.599976,1268.400024,1234.264160,5427775,937.933440
2020-01-06,1260.000000,1261.800049,1236.000000,1240.949951,1207.552979,5445093,994.922915
2020-01-07,1258.900024,1271.449951,1252.250000,1260.599976,1226.674072,7362247,1036.927180


### Display the plot showing Kalman estimate against stock price
This code creates an interactive Plotly chart with two line plots: one plotting the 'Adj Close' column and the other the 'KF_mean' column from a DataFrame 'data' against its index. The layout sets the chart's title and axis labels, and iplot(fig) displays the chart.

In [4]:
# Assuming 'data' is your DataFrame
trace1 = go.Scatter(x=data.index, y=data['Adj Close'], mode='lines', name='SPY')
trace2 = go.Scatter(x=data.index, y=data['KF_mean'], mode='lines', name='Kalman Estimate')

layout = go.Layout(title='Kalman Filter Estimates for HDFCBANK',
                   xaxis=dict(title='Day'),
                   yaxis=dict(title='Price'))

fig = go.Figure(data=[trace1, trace2], layout=layout)

iplot(fig)